In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import data trong csv
df = pd.read_csv("sales.csv")
df.head()

In [ ]:
# kiem tra null
df.isnull().sum()

In [ ]:
# sua ten + xoa cot index
df = df.iloc[:, 1:]
df = df.rename(
    columns={
        "TV Ad Budget ($)": "TV",
        "Radio Ad Budget ($)": "Radio",
        "Newspaper Ad Budget ($)": "Newspaper",
        "Sales ($)": "Sales",
    }
)
df.tail()

In [ ]:
# ve bang heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(df.corr(), annot=True, cmap=plt.cm.Reds, fmt=".2f")
plt.title("Ma trận tương quan giữa các biến")
plt.show()

In [ ]:
# tao mô hình
model = LinearRegression()
X = df[['TV', 'Radio', 'Newspaper']]
y = df['Sales']

In [ ]:
# chia tập train : test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [ ]:
# train mô hình
model.fit(X_train, y_train)

In [ ]:
# dự đoán
y_pred = model.predict(X_test)

# In kết quẳ

In [ ]:
# Công thức tính biến phụ thuộc y
coef_df = pd.DataFrame(model.coef_, X.columns, columns=['Hệ số (beta)'])
print("Hệ số hồi quy:", coef_df)
print(f"Hệ số chặn: {model.intercept_:.4f}")

# In phương trình hồi quy tuyến tính
print("\nPhương trình hồi quy tuyến tính:")
equation = f"Sales = {model.intercept_:.4f}"

for col, coef in zip(X.columns, model.coef_):
    equation += f" + ({coef:.4f})*{col}"
print(equation)

In [ ]:
# Đánh giá mô hình thông qua tập test
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.4f}")
print(f"R²: {r2:.4f}")

# so sánh thực tế
comparion = pd.DataFrame({"Thực tế": y_test, "Dự đoán": y_pred})
print(comparion.head())

In [ ]:
# 7. Vẽ biểu đồ từng biến (TV, Radio, Newspaper)
fig, axs = plt.subplots(1, 3, figsize=(18,5))

features = ['TV', 'Radio', 'Newspaper']
colors = ['purple', 'green', 'orange']

for i, col in enumerate(features):
    axs[i].scatter(X_test[col], y_test, color=colors[i], label='Thực tế')
    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Sales')
    axs[i].set_title(f'Hồi quy: {col} vs Sales')

    # Tính giá trị dự đoán tuyến tính theo cột hiện tại (giữ các biến khác ở giá trị trung bình)
    X_temp = X_test.copy()
    X_temp[[c for c in features if c != col]] = X_test[[c for c in features if c != col]].mean()
    y_line = model.predict(X_temp)
    
    # Vẽ đường hồi quy
    axs[i].plot(X_test[col], y_line, color='black', linewidth=2, label='Đường hồi quy')

    axs[i].legend()

plt.tight_layout()
plt.show()